In [ ]:
'''
Usa o grid para eXportar as imagens combinadas
As imagens já devem estar no mapa
'''

import os
import arcpy
import time

# --- 1. CONFIGURAÇÃO ---
# Use uma pasta de saída nova para este teste final
output_folder = r"C:\Users\davils\Documents\Lauriano\Code\Arcpy\Paulo\output_rasters2"

# Nome do Layout que será usado para a exportação.
layout_name = "Layout1"

# Nome do Map Frame DENTRO do Layout que será atualizado.
map_frame_name = "Map Frame"
grid_layer_name = "grid20km" 

# Nome do seu mapa no painel de Conteúdo
map_name = "Map" 

# --- 2. INICIALIZAÇÃO DA ABORDAGEM DIRETA ---
print("Iniciando SCRIPT DE EXPORTAÇÃO DIRETA DO MAPA...")
aprx = arcpy.mp.ArcGISProject("CURRENT")

# Pega o mapa pelo nome
mapa = aprx.listMaps(map_name)[0]
if not mapa:
    raise Exception(f"Mapa com o nome '{map_name}' não encontrado.")

# Pega a camada de grid dentro do mapa especificado
grid_layer = next((lyr for lyr in mapa.listLayers() if lyr.name.lower() == grid_layer_name.lower()), None)
if not grid_layer:
    raise Exception(f"A camada de grid '{grid_layer_name}' não foi encontrada no mapa '{map_name}'.")

# Pega a visualização ativa (deve ser a do mapa)
map_view = aprx.activeView

print(f"Trabalhando com o Mapa: '{mapa.name}' e a View Ativa.")
print("-" * 30)

# --- 3. PROCESSAMENTO E EXPORTAÇÃO DIRETA ---
with arcpy.da.SearchCursor(grid_layer, ["SHAPE@", "OID@"]) as cursor:
    for i, row in enumerate(cursor):
        quadrado_geom = row[0]
        quadrado_id = row[1]
        
        if quadrado_geom:
            print(f"\n--- Processando Quadrado ID: {quadrado_id} ---")
            extent = quadrado_geom.extent
            
            # **AÇÃO 1: Atualiza a câmera da Map View (não do layout)**
            map_view.camera.setExtent(extent)
            print(f"Zoom aplicado na Map View para o extent do grid ID {quadrado_id}.")
            
            # Pausa para garantir que a tela redesenhe
            time.sleep(1.5) 
            
            # **AÇÃO 2: Exporta a Map View diretamente**
            output_filename = f"export_direto_{grid_layer_name}_{quadrado_id}.tif"
            output_path = os.path.join(output_folder, output_filename)

            # A função de exportação da MapView é mais simples e robusta
            map_view.exportToTIFF(
                output_path,
                width=2000,  # Largura da imagem em pixels (ajuste se necessário)
                height=2000, # Altura da imagem em pixels (ajuste se necessário)
                world_file=True
            )
            print(f"Exportado: {output_filename}")

print("\n--- PROCESSO DE EXPORTAÇÃO DIRETA CONCLUÍDO ---")

In [ ]:
'''
Adiciona as imagens automaticamente ao mapa, com filtro para cobertura de nuvens e procurando para % dias atras
Não cheguei testar
'''
import os
from datetime import datetime, timedelta, time, timezone
# A biblioteca do Planet que acabamos de instalar
import planet 

# =======================================================================
# --- 1. CONFIGURAÇÃO ---
# =======================================================================

# COLOQUE SUA CHAVE DE API DO PLANET AQUI
# É recomendado configurar como uma variável de ambiente, mas para
# um script simples, podemos colocar diretamente aqui.
PLANET_API_KEY = "PLAKa0657b23d6d04653b63dcb1eefdaaacc"

import os
from datetime import datetime, timedelta, time, timezone
# A biblioteca do Planet que instalamos
import planet 
# Biblioteca para gerenciar a execução assíncrona
import asyncio

# Área de Interesse (AOI): Geometria do Estado do Rio de Janeiro
AOI_RIO_DE_JANEIRO = {
    "type": "Polygon",
    "coordinates": [
        [
            [-44.8, -23.4], [-44.8, -20.8], [-41.0, -20.8],
            [-41.0, -23.4], [-44.8, -23.4]
        ]
    ]
}

# Filtro máximo de cobertura de nuvens (30%)
MAX_CLOUD_COVER = 0.30

# Janela máxima de busca para trás (em dias) antes de desistir
DIAS_MAX_BUSCA = 90

# =======================================================================
# --- 2. FUNÇÃO DE BUSCA (permanece a mesma) ---
# =======================================================================

async def find_planet_image(start_date, aoi_geom, max_cloud, api_key, search_window_days):
    """
    Busca por uma imagem Planet a partir de uma data inicial, iterando para trás
    até encontrar uma que satisfaça os critérios.
    """
    print(f"Iniciando busca a partir de: {start_date.strftime('%d/%m/%Y')}")
    
    async with planet.Session(auth=planet.Auth.from_key(api_key)) as sess:
        client = sess.client('data')

        for i in range(search_window_days):
            current_date = start_date - timedelta(days=i)
            print(f"... procurando em {current_date.strftime('%d/%m/%Y')}...")

            start_time = datetime.combine(current_date, time.min).replace(tzinfo=timezone.utc)
            end_time = datetime.combine(current_date, time.max).replace(tzinfo=timezone.utc)

            geometry_filter = planet.data_filter.geometry_filter(aoi_geom)
            date_range_filter = planet.data_filter.date_range_filter('acquired', gte=start_time, lte=end_time)
            cloud_cover_filter = planet.data_filter.range_filter('cloud_cover', lte=max_cloud)

            combined_filter = planet.data_filter.and_filter([
                geometry_filter,
                date_range_filter,
                cloud_cover_filter
            ])
            
            item_types = ["PSScene"]

            search_request = await client.create_search(name="busca_rj", search_filter=combined_filter, item_types=item_types)
            found_items = [item async for item in client.search(search_id=search_request['id'], limit=1)]
            
            if found_items:
                found_item = found_items[0]
                item_id = found_item['id']
                acquired_date = found_item['properties']['acquired']
                cloud_percent = found_item['properties']['cloud_cover'] * 100
                
                print("\n✅ IMAGEM ENCONTRADA!")
                print(f"   - ID do Item: {item_id}")
                print(f"   - Data da imagem: {acquired_date.split('T')[0]}")
                print(f"   - Cobertura de nuvem: {cloud_percent:.2f}%")
                return found_item

        print("\n❌ Nenhuma imagem encontrada dentro da janela de busca.")
        return None

# =======================================================================
# --- 3. FUNÇÃO PRINCIPAL PARA GERENCIAR AS CHAMADAS ---
# =======================================================================

async def main():
    """
    Função principal que organiza e 'await' as buscas de forma correta.
    """
    # Pega a data de hoje (importante usar UTC para consistência com a API)
    today_utc = datetime.now(timezone.utc).date()

    # --- Busca 1: Imagem mais recente ---
    print("---")
    print("1️⃣  Procurando a imagem mais recente (a partir de hoje)")
    print("---")
    imagem_recente = await find_planet_image(
        start_date=today_utc,
        aoi_geom=AOI_RIO_DE_JANEIRO,
        max_cloud=MAX_CLOUD_COVER,
        api_key=PLANET_API_KEY,
        search_window_days=DIAS_MAX_BUSCA
    )

    # --- Busca 2: Imagem de 5 dias atrás ---
    print("\n" + "="*50 + "\n")
    data_alvo_2 = today_utc - timedelta(days=5)
    print("---")
    print(f"2️⃣  Procurando a segunda imagem (a partir de {data_alvo_2.strftime('%d/%m/%Y')})")
    print("---")
    imagem_antiga = await find_planet_image(
        start_date=data_alvo_2,
        aoi_geom=AOI_RIO_DE_JANEIRO,
        max_cloud=MAX_CLOUD_COVER,
        api_key=PLANET_API_KEY,
        search_window_days=DIAS_MAX_BUSCA
    )

# =======================================================================
# --- 4. EXECUÇÃO DO SCRIPT ---
# =======================================================================
# Agora, rodamos a função 'main' que gerencia as chamadas 'await'.
# Isso garante que o código assíncrono seja executado corretamente.
asyncio.run(main())